In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import glob
import mmmpy
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#from cartopy.io.img_tiles import StamenTerrain
#import pygrib
import os
#import pyart
%matplotlib inline

In [2]:
files = glob.glob("/network/rit/home/hb533188/dgx_basulab/Harish/Gust_field_nowcasting_from_Sparse_stations/data/MRMS_grib_data/CONUS/MergedReflectivityQCComposite_00.50/20201014/*.grib2.gz", recursive=True)
files

[]

In [3]:
files = glob.glob("data/MRMS_grib_data/CONUS/MergedReflectivityQCComposite_00.50/20201014/*.grib2.gz", recursive=True)
files

[]